In [2]:
import pandas as pd

In [3]:
data=pd.read_csv("processed_data.csv").drop(columns=['File_Path'])

In [4]:
data['Region/Authority']=data['Link'].str.extract(r'/([^/]+)(?=\.pdf)')
data['Region/Authority'] = data['Region/Authority'].str.lower()
data['Region/Authority'] = data['Region/Authority'].str.capitalize()


In [5]:
data

,Year,Title,Link,Extracted_Text,Region/Authority
0,2024,"IMFC Statement by Christine Lagarde, President...",https://meetings.imf.org/-/media/AMSM/Files/AM...,\r\n \r\n INTERNATIONAL MONETARY AND FINANCIA...,Ecb
1,2024,"IMFC Statement by HE Haitham Al Ghais, Secreta...",https://meetings.imf.org/-/media/AMSM/Files/AM...,\r\n \r\n INTERNATIONAL MONETARY AND FINANCIA...,Opec
2,2024,"IMFC Statement by Ayman Al-Sayari, Governor of...",https://meetings.imf.org/-/media/AMSM/Files/AM...,\r\n \r\n INTERNATIONAL MONETARY AND FINANCIA...,Saudi
3,2024,"IMFC Statement by Antoine Armand, Minister of ...",https://meetings.imf.org/-/media/AMSM/Files/AM...,INTERNATIONAL MONETARY AND FINANCIAL COMMITTE...,France
4,2024,"IMFC Statement by Luis Caputo, Minister of Eco...",https://meetings.imf.org/-/media/AMSM/Files/AM...,\r\n \r\n INTERNATIONAL MONETARY AND FINANCIA...,Argentina
...,...,...,...,...,...
559,2004,IMFC Statement by the Honorable Domenico Sinis...,https://www.imf.org/External/AM/2004/imfc/stat...,\r\n International Monetary and \r\nFinancial...,Itae
560,2004,"IMFC Statement by the Honorable John W. Snow, ...",https://www.imf.org/External/AM/2004/imfc/stat...,\r\n International Monetary and \r\nFinancial...,Usae
561,2004,IMFC Statement by H.E. Sadakazu Tanigaki Minis...,https://www.imf.org/External/AM/2004/imfc/stat...,\r\n International Monetary and \r\nFinancial...,Jpne
562,2004,"IMFC Statement By James D. Wolfensohn, Preside...",https://www.imf.org/External/AM/2004/imfc/stat...,\r\n International Monetary and \r\nFinancial...,Wbe


In [6]:
all=list(data['Region/Authority'])

In [7]:
all=list(dict.fromkeys(all))

In [8]:
constituency_map = {
    "G7": [
        "USA", "Canada", "United Kingdom", "France", "Germany", "Italy", "Japan"
    ],
    "G20": [
        "Argentina", "Brazil", "China", "India", "Indonesia", "Saudi Arabia", "Russia", "South Africa",
        "Turkey", "USA", "United Kingdom", "Germany", "Japan", "Australia", "Mexico", 
        "South Korea", "European Union"
    ],
    "OECD": [
        "Australia", "Canada", "France", "Germany", "Italy", "Japan", "Netherlands", 
        "Norway", "Spain", "United Kingdom", "United States", "Sweden"
    ],
    "OPEC": [
        "Saudi Arabia", "Algeria", "Gabon", "Nigeria", "Angola", "Iran"
    ],
    "WTO": [
        "China", "India", "USA", "European Union"
    ],
    "UN": [
        "United Nations", "International Labour Organization (ILO)", "United Nations Conference on Trade and Development (UNCTAD)"
    ],
    "EU": [
        "France", "Germany", "Italy", "Spain", "Netherlands", "Austria", "Belgium", 
        "Finland", "Sweden", "Norway"
    ],
    "Other": [
        "Djibouti", "Djibouti-fr", "Peoples Republic of China", "Democratic Republic of the Congo",
        "Gabon", "Malaysia", "Singapore", "Belgium", "Côte d'Ivoire", "South Korea", "Turkey"
    ],
    # Add abbreviations
    "Abbreviations": {
        "Ecb": "European Central Bank", 
        "Opec": "Organization of the Petroleum Exporting Countries", 
        "Un": "United Nations",
        "Tur": "Turkey", "Sau": "Saudi Arabia", "Jpn": "Japan", "Dza": "Algeria", 
        "Can": "Canada", "Aus": "Australia", "Ita": "Italy", "Bra": "Brazil", "Arg": "Argentina",
        "Nld": "Netherlands", "Fra": "France", "Che": "Switzerland", "Zaf": "South Africa",
        "Cog": "Congo", "Nor": "Norway", "Esp": "Spain", "Deu": "Germany", "Rus": "Russia", 
        "Ind": "India", "Gbr": "United Kingdom", "Tha": "Thailand", "Nga": "Nigeria", 
        "Are": "United Arab Emirates", "Wto": "World Trade Organization", "Bel": "Belgium", 
        "Ecofin": "Economic and Financial Affairs Council", "Fin": "Finland", "Chn": "China",
        "Swe": "Sweden", "Kor": "South Korea", "Cod": "Côte d'Ivoire", "Cze": "Czech Republic", 
        "Per": "Peru", "Idn": "Indonesia", "Est": "Estonia", "Cmr": "Cameroon", "Sgp": "Singapore", 
        "Bfa": "Burkina Faso", "Dnk": "Denmark", "Mys": "Malaysia", "Gab": "Gabon", "Col": "Colombia", 
        "Fsb": "Financial Stability Board", "Aut": "Austria", "Ltu": "Lithuania", "Eu": "European Union", 
        "Chl": "Chile", "Wb": "World Bank", "Fsf": "Financial Stability Fund", "Imf": "International Monetary Fund",
        "Imfe": "International Monetary Fund (Extended Fund Facility)", "Uaee": "United Arab Emirates", 
        "Gbre": "United Kingdom", "Inde": "India", "Ause": "Australia", "Thae": "Thailand", 
        "Deue": "Germany", "Nore": "Norway", "Cane": "Canada", "Ruse": "Russia", "Chee": "Switzerland", 
        "Bele": "Belgium", "Itae": "Italy", "Usae": "United States", "Jpne": "Japan", "Wbe": "World Bank", 
        "Eue": "European Union"
    }
}

# Helper function to find the constituency of each country/authority
def map_to_constituency(authorities):
    result = {}
    for authority in authorities:
        # First, check if it's an abbreviation in the "Abbreviations" dictionary
        full_name = constituency_map["Abbreviations"].get(authority, authority)
        
        # Check each constituency for membership
        found = False
        for constituency, members in constituency_map.items():
            if constituency != "Abbreviations" and full_name in members:
                result[authority] = constituency
                found = True
                break
        
        if not found:
            result[authority] = "Unknown"  # In case it doesn't match any constituency
    
    return result

In [9]:
# Map authorities to their constituencies
constituency_result = map_to_constituency(all)

# Print the mapping
for authority, constituency in constituency_result.items():
    print(f"{authority}: {constituency}")

Ecb: Unknown
Opec: Unknown
Saudi: Unknown
France: G7
Argentina: G20
Australia: G20
Oecd: Unknown
Spain: OECD
Ec: Unknown
Italy: G7
Brazil: G20
Netherlands: OECD
Austria: EU
Japan: G7
Switzerland: Unknown
Sa: Unknown
Germany: G7
Uae: Unknown
Gabon: OPEC
China: G20
Uk: Unknown
Russia: G20
India: G20
Thailand: Unknown
Algeria: OPEC
Eucouncil: Unknown
Norway: OECD
Usa: Unknown
Ghana: Unknown
Djibouti: Other
Djibouti-fr: Other
Saudi-arabia: Unknown
Eu-council-of-economic-and-finance-ministers: Unknown
Chile: Unknown
European-commission: Unknown
Canada: G7
Ilo: Unknown
United-kingdom: Unknown
Switzterland: Unknown
Finland: EU
United-nations: Unknown
Belgium: EU
Singapore: Other
United-states: Unknown
Nigeria: OPEC
United-arab-emirates: Unknown
Korea: Unknown
Cote-divoire: Unknown
Civ-f: Unknown
Sweden: OECD
Canada-f: Unknown
Unctad: Unknown
Kingdom-of-the-netherlands: Unknown
Hungary: Unknown
Indonesia: G20
Peoples-republic-of-china: Unknown
Republic-of-korea: Unknown
Democratic-republic-of-